In [1]:
# !pip3 install arxiv
# !pip3 install wikipedia

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=427c3bb202ab58109b52d5f8e22b9ac6429cf013f1e904dd20966c98257b6e67
  Stored in directory: c:\users\anjana\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [35]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [36]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki_tool.name


'wikipedia'

In [37]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings


In [38]:
# loader = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")
loader = TextLoader("sample.txt")
doc = loader.load()
chunk_doc = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(doc)


In [39]:
chunk_doc 

[Document(metadata={'source': 'sample.txt'}, page_content='LangChain is the framework for building context-aware reasoning applications')]

In [40]:
vectordb = FAISS.from_documents(chunk_doc, OllamaEmbeddings())
vect_retriever = vectordb.as_retriever() 

In [41]:
from langchain.tools.retriever import create_retriever_tool


In [42]:
vect_db_tool = create_retriever_tool(vect_retriever, "lanchain_Search",
                                       "Search for any information about langchain")

In [43]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results = 1, 
                                doc_content_chars_max=200)

arxiv_tool= ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [44]:
tools = [vect_db_tool, wiki_tool, arxiv_tool]
tools 

[Tool(name='lanchain_Search', description='Search for any information about langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001BC7C628900>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BC6C8E8D10>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001BC7C6B9120>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BC6C8E8D10>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\anjana\\AppData\\L

In [45]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

In [46]:
# from langchain import hub
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(input_variables=["input", "agent_scratchpad"], template="Answer the question using available tools: {input} and {agent_scratchpad}")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='Answer the question using available tools: {input} and {agent_scratchpad}')

In [47]:
## from langchain.tools.retriever import create_retriever_tools_agent
from langchain.agents import create_openai_tools_agent

# from langchain.agents import initialize_agent
agent = create_openai_tools_agent(llm, tools, prompt)

# agent = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent_type="zero-shot-react-description",  # This allows the agent to call tools reactively
#     prompt=prompt
# )


In [48]:
from langchain.agents import AgentExecutor
retriever = AgentExecutor(agent=agent, tools = tools, verbose=True)

In [49]:
retriever.invoke({"input": "What is Langchain"})  



> Entering new AgentExecutor chain...
